In [1]:
from YouTube import YouTubeManager
from filesManager import filesManager
from app import app
from response import response


# from itertools import count
from pathlib import Path
from collections import defaultdict
import pandas as pd

import time, re
files_manager = filesManager()
yt = YouTubeManager()
functions = app()
response_mnr = response()
quota = files_manager.get_today_quota(True)
pd.set_option('display.max_rows', None)

The current quota usage is 0


In [2]:
YT_content_creators_iter = functions.get_df_to_iterate(files_manager.playlist_folder, files_manager.YT_content_creators)
YT_content_creators_iter

,Handle,channelName,channelID,uploadsID
0,a24,A24,UCuPivVjnfNo4mb3Oog_frZg,UUuPivVjnfNo4mb3Oog_frZg
1,aagulla,Andres Agulla,UC7LMKjP8uqyRIjJGr44SY4A,UU7LMKjP8uqyRIjJGr44SY4A
2,abbyandersonmusic,Abby Anderson,UCQUsUoxjKwuVI3txJnYK2rA,UUQUsUoxjKwuVI3txJnYK2rA
3,academiaplay,Academia Play,UCv05qOuJ6Igbe-EyQibJgwQ,UUv05qOuJ6Igbe-EyQibJgwQ
4,adele,Adele,UCsRM0YB_dabtEPGPTKo-gcw,UUsRM0YB_dabtEPGPTKo-gcw
5,agujerosdeguion,Agujeros de Guion,UCffvdhj3nA0s2uN3NoWDQFA,UUffvdhj3nA0s2uN3NoWDQFA
6,alairecon,Al Aire Con Coello y Llaca,UCpzZ0nMQLooAHkzh19NfHKA,UUpzZ0nMQLooAHkzh19NfHKA
7,alexandradaddario,Alexandra Daddario,UCIU3ZZKZxT5e7OBA5avh5VQ,UUIU3ZZKZxT5e7OBA5avh5VQ
8,alicaschmidt4882,Alica Schmidt,UCcE1cmPLAh2g7lVo1s4n22g,UUcE1cmPLAh2g7lVo1s4n22g
9,alicemerton,Alice Merton,UC-kzn5xgDaXXRi_pvcZ02rQ,UU-kzn5xgDaXXRi_pvcZ02rQ


In [3]:
yt_channel = 'https://www.youtube.com/channel/'
yt_url = 'https://www.youtube.com/watch?v='

In [4]:
subscriptions =  yt.get_subscriptions()
subcribed = []
for sub in subscriptions:
    snippet = sub.get('snippet', {}) # dict_keys(['publishedAt', 'title', 'description', 'resourceId', 'channelId', 'thumbnails'])
    publishedAt = snippet.get('publishedAt', yt.default_date)
    resourceId = snippet.get('resourceId', "")
    title = snippet.get('title', '')
    channelId = resourceId.get('channelId', '')
    subcribed.append(
        {'publishedAt': publishedAt, #publishedAt represents the date that you subscribed to the Channel.
        'title': title,
        'channelId': channelId}
    )
if subcribed:
    print('Channels that are not in the Data Frame')

    alignment = max(len(sub['title']) for sub in subcribed)
    counter = 1
    for sub in subcribed:
        if sub['channelId'] not in files_manager.YT_content_creators.channelID.values:
            print(f"{counter:03d} {sub['title']:<{alignment}} {yt_channel}{sub['channelId']}")
            counter+=1
            

Channels that are not in the Data Frame
001 Seinfeld                                    https://www.youtube.com/channel/UCKv01ZGOJhp2HbtDe4EvS8A
002 Call me Back Podcast - with Dan Senor       https://www.youtube.com/channel/UCxDmjhiSTRbCE9QOFSKmykg
003 Warner Bros. Entertainment                  https://www.youtube.com/channel/UCgKkNPU2Ib7_TcyAl8M2S-w
004 The Saiyan Kiwi                             https://www.youtube.com/channel/UCxNkYaBMoWZ5_P0KDJl5UUw
005 Bridget Phetasy                             https://www.youtube.com/channel/UC5SQnm72FspDNUXK4cB2_xg
006 Warner Bros. TV                             https://www.youtube.com/channel/UCS1MgGMoPaagPOzV1EHknhw
007 Nickelback                                  https://www.youtube.com/channel/UCz8FPpkJMwayyReSDLTX8IQ
008 ENRIQUE GARAY                               https://www.youtube.com/channel/UC1QoYWu_Db6jZpH328LNKhg
009 Calixpert                                   https://www.youtube.com/channel/UCXrmkB1MeLXR9Thv8RHBVGA
010 National Re

In [5]:
alignment

43

# **Create Dictionary of the Playlists**

In [6]:
playlist_names = yt.get_all_playlists()
youtube_names = [file.stem.replace('_', ' ').strip() for file in files_manager.playlist_folder.iterdir() if file.suffix == '.txt']
youtube_playlists = defaultdict(lambda:
                                  {"Handles":[],
                                   "Playlist_ID": "",
                                   "video_ids": [],
                                   "new_video_ids": [],
                                   # "new_videos_duration": 0.0,
                                  }
                                  )


shorts_playlist_name = 'Shorts To Watch'
other_playlist_name = 'Videos To Watch'
Joey_playlist = 'Joey'
WL_shorts_playlist = 'Watch Later Shorts'
TBBT_playlist = 'The Official BBT Podcast'
vertical_video_id = 'dHtSz14yQe8'
special_playlist = [other_playlist_name, Joey_playlist, TBBT_playlist, WL_shorts_playlist, shorts_playlist_name]
# youtube_names.extend({'Path': None, 'Name': playlist} for playlist in special_playlist)
youtube_names.extend(special_playlist)

for playlist in youtube_names:
    file_path = files_manager.playlist_folder / f'{playlist.replace(" ","_")}.txt'
    if file_path.exists():
        handles = file_path.read_text(encoding="utf-8").splitlines()
        youtube_playlists[playlist]['Handles'].extend(handles) 

    #Getting the Playlist ID
    playlist_id = next((d["id"] for d in playlist_names if playlist in d.values()), None)
    youtube_playlists[playlist]['Playlist_ID'] = playlist_id

    if playlist_id:
        video_ids = yt.get_all_ids_playlist(playlist_id, 300)
        youtube_playlists[playlist]['video_ids'].extend(video_ids)

all_ids_from_playlist = [video_id for playlist in youtube_playlists.values() for video_id in playlist['video_ids']]
print(f'There are {len(all_ids_from_playlist):,} videos in all the playlists')

Retrieved all playlists successfully!
There are 1,753 videos in all the playlists


# **Exceptions**

In [7]:
skip_handle_shorts_path = files_manager.exception_folder / 'skip_shorts_handle.txt'
skip_shorts_handle = files_manager.get_elements_from_file(skip_handle_shorts_path)

skip_long_videos_60m_path = files_manager.exception_folder / 'skip_long_videos_60m.txt'
skip_long_videos = files_manager.get_elements_from_file(skip_long_videos_60m_path, create_file = True)

skip_live_handle = files_manager.exception_folder /'skip_live_handle.txt'
skip_liveStreamingDetails_handle = files_manager.get_elements_from_file(skip_live_handle, create_file = True)

skip_title_path = files_manager.exception_folder /'skip_title.txt'
titles_list = files_manager.get_elements_from_file(skip_title_path, create_file = True)

only_add_long_videos_path = files_manager.exception_folder / 'only_add_long_videos.txt'
only_long_videos = files_manager.get_elements_from_file(only_add_long_videos_path, create_file = True)

WL_shorts_path = files_manager.exception_folder / 'WL_shorts.txt'
WL_shorts = files_manager.get_elements_from_file(WL_shorts_path, create_file = True)


# **Add to files all the Video IDs that were added manually to any playlist**

In [8]:
missing_video_ids = files_manager.find_missing_elements(all_ids_from_playlist)
print(len(missing_video_ids))
saved_quota = 0
manually_added = defaultdict(list)

for video_id in missing_video_ids:
    response = yt.get_response_video_id(video_id)
    items = response.get('items',[])
    if not items:
        continue
    snippet = items[0].get('snippet', {})
    channelId = snippet.get('channelId', "")
    if channelId in files_manager.YT_content_creators['channelID'].values:
        handle = files_manager.YT_content_creators[files_manager.YT_content_creators['channelID'] == channelId]['Handle'].iloc[0]
        handle_path = files_manager.content_creator_folder / f'{handle}.txt'
        files_manager.add_element_to_file(handle_path, video_id, sort_list = False, print_statement = True)
        manually_added[handle].append(response)
        saved_quota += 50
    else:
        print(f'{yt_url}{video_id}')
print(f'The saved quota was: {saved_quota:,}')

0
The saved quota was: 0


In [9]:
if manually_added:
    print('Videos IDs that were manually added to any playlist')
    for handle in manually_added:
        responses = manually_added[handle]
        print(handle)
        for response in responses:
            video_info = response_mnr.get_video_info(response)
            print(f"\t{video_info['title']}")
            print(f"\t{video_info['publishedAt']}")
            print(f"\t{yt_url}{video_info['video_id']}")
            print('*'*50)
        print('-'*50)

# **Get the new IDs**

In [10]:
was_braked = False
num_rows = len(YT_content_creators_iter)
digits = len(str(num_rows))
message = ''
missing_handles = []
ids_with_error = []
livestreams = []
start = time.time()
for row in YT_content_creators_iter.itertuples():
    if was_braked:
        break
    handle = row.Handle
    channelName = row.channelName
    channelID = row.channelID
    uploadsID = row.uploadsID

    print(" "*len(message), end='\r')
    message = f'{row.Index + 1:0{digits}d} / {num_rows}: {channelName}'
    print(message, end='\r')
    videos_ids = yt.get_all_ids_playlist(uploadsID, 1)
    videos_ids.reverse()
    file_path = files_manager.content_creator_folder / f'{handle}.txt'
    handle_ids = files_manager.get_elements_from_file(file_path, create_file = True)
    playlist_key = next((key for key, handles in youtube_playlists.items() if handle in handles.get('Handles',[])), None)

    for video_id in videos_ids:
        if video_id not in handle_ids:# or video_id not in all_ids_from_playlist:
            response = yt.get_response_video_id(video_id)
            video_id_info = response_mnr.get_video_info(response)
            video_id_info['file_path'] = file_path
            video_id_info['response'] = response
            if not video_id_info:
                continue            
            
            elif video_id_info['liveBroadcastContent'] == 'upcoming' or  video_id_info['duration'] == 0:
                continue
            elif response_mnr.is_restricted(response):
                files_manager.add_element_to_file(file_path, video_id, False)
                response_mnr.add_response_df(files_manager.restriction_folder / f'{handle}.csv', response)
                
            elif video_id_info['liveStreamingDetails'] and handle in skip_liveStreamingDetails_handle:
                files_manager.add_element_to_file(file_path,video_id, False)
                livestreams.append(video_id)  
            
            elif (handle in only_long_videos and duration < 35*60) or \
            any(functions.remove_accents(t.lower()) in functions.remove_accents(video_id_info["title"].lower()) for t in titles_list) or \
            (handle in skip_long_videos and video_id_info['duration'] >= 60*60) or video_id_info['duration'] >= 60*60*3:
                files_manager.add_element_to_file(file_path,video_id, False)
            else:
                short = functions.is_short(video_id)
                if short is None:
                    was_braked = True
                    break
                    
                elif short is True:
                    if handle in WL_shorts:
                        youtube_playlists[WL_shorts_playlist]['new_video_ids'].append(video_id_info)
                    elif handle not in skip_shorts_handle:
                        youtube_playlists[shorts_playlist_name]['new_video_ids'].append(video_id_info)
                    else:
                        files_manager.add_element_to_file(file_path,video_id, False)
                elif playlist_key:
                    youtube_playlists[playlist_key]['new_video_ids'].append(video_id_info)
                else:
                    youtube_playlists[other_playlist_name]['new_video_ids'].append(video_id_info)

    if was_braked:
        break                 
            
print(" "*len(message), end='\r')
print(f'Duration to getting the new IDs => {functions.duration_string(time.time() - start)}')
if not was_braked:
    total_duration = sum(video_id['duration'] for playlist in youtube_playlists for video_id in youtube_playlists[playlist]['new_video_ids'])
    print(f"Duration of all the new Video IDs => {functions.duration_string(total_duration)}")

    total_videos = sum(len(youtube_playlists[playlist]['new_video_ids']) for playlist in youtube_playlists)
    print(f'There are {total_videos} total videos to add')
else:
    print('There is an error with the request function. You might be blocked')

Duration to getting the new IDs => 00:02:45          
Duration of all the new Video IDs => 03:46:59
There are 38 total videos to add


In [11]:
quota_limit = 9000
was_breaked = False
message = ''
added_videos = defaultdict(list)
quota_i = files_manager.get_today_quota(False)
for playlist in youtube_playlists:
    new_video_ids = youtube_playlists[playlist]['new_video_ids']
    playlist_id = youtube_playlists[playlist]["Playlist_ID"]
    new_video_ids.sort(key= lambda x: x['publishedAt'])
    if files_manager.get_today_quota() > quota_limit:
        was_breaked = True
        break
    if new_video_ids:
        if not playlist_id:
            response_playlist = yt.create_private_playlist(playlist, playlist)
            playlist_id = response_playlist.get('id', "")
            youtube_playlists[playlist]["Playlist_ID"] = playlist_id
            if not playlist_id:
                continue
        for index, video_info in enumerate(new_video_ids, 1):
            file_path = video_info['file_path']
            video_id = video_info['video_id']
            print(' '*len(message), end='\r')
            message = f'Adding {video_id} from {file_path.stem} to the playlist {playlist}'
            print(message, end='\r')
            if files_manager.get_today_quota() > quota_limit:
                was_breaked = True
                break
            if video_id not in youtube_playlists[playlist]['video_ids'] and yt.add_video_to_playlist(playlist_id, video_id):
                files_manager.add_element_to_file(file_path, video_id, False, False)
                youtube_playlists[playlist]['video_ids'].append(video_id)
                added_videos[playlist].append(video_info)
print(' '*len(message), end='\r')         
if was_breaked:
    print(f'The process was interruped. The last video is was {video_id} from {file_path.name}')
consumed_quota = files_manager.get_today_quota(False) - quota_i
print(f'It was consumed {consumed_quota:,} quotas in the adding process')


It was consumed 1,900 quotas in the adding process                               


In [12]:
alignment = max(len(playlist) for playlist in added_videos)
# sorted_keys = sorted(added_videos, key=lambda x:len(added_videos[x]), reverse=True)
sorted_keys = sorted(added_videos, key=lambda x:sum(video['duration'] for video in added_videos[x]), reverse=True)
ansi_pattern = re.compile(r'\x1b\[[0-9;]*m')  

val_alignment = 2
for playlist in sorted_keys:
    duration = sum(video['duration'] for video in added_videos[playlist])
    num_videos = len(added_videos[playlist])
    bold_key = f"\033[1;4m{playlist}:\033[0m" 
    extra_alignmnet = len(bold_key) - len(ansi_pattern.sub('', bold_key)) + 1

    print(f'{bold_key:<{alignment + extra_alignmnet}} {num_videos:>{2}} {functions.duration_string(duration)}')


Sports:                   2 00:47:47
TV Shows & Movies Clips:  9 00:41:44
Podcasts:                 3 00:40:14
Music to Like:            2 00:23:45
French Like:              1 00:20:04
Shorts To Watch:         13 00:16:28
Videos To Watch:          1 00:14:01
Fitness:                  1 00:10:40
Cine:                     1 00:08:56
Actualites et cinema:     1 00:02:13
Watch Later Shorts:       4 00:01:07


In [13]:
quota = files_manager.get_today_quota(True)  

The current quota usage is: 2,404
